In [1]:
import numpy as np
import pandas as pd


In [2]:
train = pd.read_csv("E:/Projects/DeepLearning/sentiment.tsv", delimiter = "\t")

In [3]:
train.head(10)

,Sentiment,Sentiment_text
0,neg,"@jamielewislewis i cant believe it, it really ..."
1,pos,having a vodka tonic and looking forward to go...
2,pos,@ddlovatofans1neg1 Could you follow me please....
3,pos,@jordanknight for once.................. PLEAS...
4,neg,Had a dream about a walk in fast food resturau...
5,pos,@TroyBrownBBNews Yes... For a &quot;friend&quot;
6,pos,Packing for work expierance
7,neg,hates @internet @explrer (angry)(angry) **but ...
8,neg,@federalcase I said I go out for eat 5:negneg...
9,neg,@babykates7 yeah they won't do the surgery til...


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2001 entries, 0 to 2000
Data columns (total 2 columns):
Sentiment         2001 non-null object
Sentiment_text    2001 non-null object
dtypes: object(2)
memory usage: 31.3+ KB


In [5]:
# Isolate target data
z = train['Sentiment']
y = np.where(z == 'pos',1,0)


In [6]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
count_vect = CountVectorizer()
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer


class LemmaTokenizer(object):
     def __init__(self):
        self.wnl = WordNetLemmatizer()
     def __call__(self, doc):
        return [self.wnl.lemmatize(t, pos='v') for t in word_tokenize(doc)]
class Lansta(object):
     def __init__(self):
        self.wnl = LancasterStemmer()
     def __call__(self, doc):
        return [self.wnl.stem(t) for t in word_tokenize(doc)]

In [161]:
import re


class processTweet(object):
    def __call__(self, tweet):
    # process the tweets

    #Convert to lower case
       self.tweet = tweet.lower()
    #Convert www.* or https?://* to URL
       self.tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',tweet)
    #Convert @username to AT_USER
       self.tweet = re.sub('@[^\s]+',' ',tweet)
    #Remove additional white spaces
       self.tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
       self.tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
     #trim
       self.tweet = tweet.strip('\'"')
        
       #self.tweet = scale(tweet)
    
       return tweet

In [132]:
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, chi2
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import LinearSVC

In [179]:
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer(tokenizer = Lansta(),  binary = True, stop_words = {'english'}, min_df = 4)), 
                     ('tfidf', TfidfTransformer(norm = 'l2', use_idf=True)),
                     ('chi2', SelectKBest(chi2, k= 500)),
                     ('clf', LogisticRegression(class_weight = 'balanced'))])

In [180]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score


kf = KFold(n_splits = 10, shuffle = True, random_state = 42)

ypred=np.zeros_like(y,dtype=float)

def run(X, y, model):
    for train_index, test_index in kf.split(X):
        #print("TRAIN:", train_index, "TEST:", test_index)
        x_train, y_train = X[train_index], y[train_index]
        x_test, y_test = X[test_index], y[test_index]
        
        model.fit(x_train, y_train)
        
        ypred[test_index]= model.predict_proba(x_test)[:,1]
        
        
    return ypred

run(train['Sentiment_text'], y , model =text_clf)

score = roc_auc_score(y, ypred)
score

0.79225288780526792

In [ ]:
ypred

In [ ]:
SGDClassifier(loss='log', penalty='l2',alpha=1e-3, n_iter=9, random_state=23)